In [ ]:
import pandas as pd
data= pd.read_csv(r"C:\Users\saras\Documents\School\STA 6704- Data Mining 2\widsdatathon2020\training_v2.csv")
data.head()

In [ ]:
data.shape

In [ ]:
pd.set_option('display.max_columns', None)
data.describe()

In [ ]:
pd.set_option('display.max_rows', None)
data.isnull().sum()
#To be noted that the target variable hospital_death do not have any missing values

In [ ]:
#Dropping columns causing target leakage
data_new= data.drop(['apache_4a_hospital_death_prob','apache_4a_icu_death_prob','heart_rate_apache','d1_heartrate_max','d1_heartrate_min','h1_heartrate_max'],axis=1)

#Dropping columns with more than 15% missing values
data_new=data_new[data_new.columns[data_new.isnull().sum()<(0.15*data_new.shape[0])]]

#Dropping unimportant columns by intuition
data_new= data_new.drop(['encounter_id','patient_id','hospital_id'],axis=1)

data_new.shape

In [ ]:
#Dropping the rows with null values and then check the distribution
data_new=data_new.dropna(axis=0)
print(data_new.shape)
data_new.describe()

In [ ]:
#Dropping the columns with only 0 in them
data_new=data_new.drop(['gcs_unable_apache','readmission_status'], axis=1)
data_new.shape

In [ ]:
#Checking the different levels and its count
col=data_new.select_dtypes(include=['object']).columns
for col in col:
    print(data_new[col].value_counts())

In [ ]:
#Combining levels of selected columns
data_new['icu_admit_source']=data_new['icu_admit_source'].replace({'Other ICU':'other','Other Hospital':'other'})
data_new['apache_2_bodysystem']=data_new['apache_2_bodysystem'].replace({'Undefined diagnoses':'Undefined Diagnoses'})
data_new['ethnicity']=data_new['ethnicity'].replace({'Asian':'Other/Unknown','Native American':'Other/Unknown','Hispanic':'Other/Unknown'})

col=data_new.select_dtypes(include=['object']).columns
for col in col:
    print(data_new[col].value_counts())

In [ ]:
!pip install scikit-learn==1.2.2

In [ ]:
pip install imbalanced-learn

In [ ]:
#Oversampling as the data is imbalanced
from imblearn.over_sampling import RandomOverSampler
X = data_new.drop(['hospital_death'], axis=1)
y = data_new['hospital_death']
over_sampler = RandomOverSampler(sampling_strategy='minority', random_state=42)
X_resampled, y_resampled = over_sampler.fit_resample(X, y)
oversampled_df = pd.DataFrame(X_resampled, columns=X.columns)
print(y_resampled.value_counts())
oversampled_df.shape

In [ ]:
#Splitting data
X=oversampled_df.drop('h1_heartrate_min',axis=1)
y=oversampled_df['h1_heartrate_min']  #Target variable

other_columns_train= X.select_dtypes(exclude=['int64', 'float64']).columns
type(other_columns_train)
#Adding all the binary and categorical variable to go ahead with encoding
other_columns_train=other_columns_train.union(['elective_surgery','apache_post_operative','arf_apache','intubated_apache','ventilated_apache','aids','cirrhosis',
'diabetes_mellitus','hepatic_failure','immunosuppression','leukemia','lymphoma','solid_tumor_with_metastasis'])
print(other_columns_train)
#Encoding categorical variables
X_encoded=pd.get_dummies(X,columns=other_columns_train,drop_first=True)
print(X_encoded.head())
print(X_encoded.shape)

In [ ]:
#from sklearn.model_selection import train_test_split
#X_train,X_test,y_train,y_test= train_test_split(X_encoded,y,test_size=0.33)

In [ ]:
#X_train.shape

In [ ]:
#Scaling the data using standardization
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()

numeric_columns = X_encoded.select_dtypes(exclude=['uint8']).columns
categoric_columns=X_encoded.select_dtypes(include=['uint8']).columns

scaled_X=scaler.fit_transform(X_encoded[numeric_columns])


                                                                                                                                                      
##scaled_X_train=pd.DataFrame(scaled_X_train,columns=X_train[numeric_columns_train].columns)
##scaled_X_train = scaled_X_train.reset_index(drop=True)
##scaled_X_train= pd.concat([scaled_X_train,X_train[other_columns_train]],axis=1)
print(scaled_X.shape)
scaled_X.shape

In [ ]:
#SApplyinh PCA for feature generation and dimesnionality reduction
from sklearn.decomposition import PCA
import numpy as np
pca = PCA()
pca.fit(scaled_X)
var1=np.cumsum(np.round(pca.explained_variance_ratio_, decimals=4)*100)
print (var1)

In [ ]:
#Plotting the cumulative variance by PCs
import matplotlib.pyplot as plt
plt.figure(figsize=(8, 6))
plt.plot(range(1, len(var1) + 1), var1, marker='o', linestyle='-', color='b',markersize=3, linewidth=1.5)
plt.xlabel('Number of Principal Components')
plt.ylabel('Cumulative Explained Variance (%)')
plt.title('Cumulative Explained Variance by Principal Components')
plt.grid(True)
plt.show()

In [ ]:
#Selecting variables
pca = PCA(n_components=7)
pca.fit(scaled_X)
X_pc=pca.fit_transform(scaled_X)
pc_columns = [f'PC{i+1}' for i in range(X_pc.shape[1])]
X_pca = pd.DataFrame(data=X_pc, columns=pc_columns)
print(X_pca.shape)
X_pca.head()

In [ ]:
print(X_encoded[categoric_columns].shape)
print(X_pca.shape)
X_pca= pd.concat([X_pca.reset_index(drop=True),X_encoded[categoric_columns].reset_index(drop=True)],axis=1,join='outer',ignore_index=False)
print(X_pca.shape)
X_pca.head()

In [ ]:
from sklearn.model_selection import train_test_split
X_train_pca,X_test_pca,y_train,y_test= train_test_split(X_pca,y,test_size=0.33)
print(X_train_pca.shape)
print(X_test_pca.shape)
print(y_train.shape)
print(y_test.shape)

In [ ]:
#Fitting decision tree with hyperparameter tuning using GridSearchCV
from sklearn.tree import DecisionTreeRegressor
from sklearn.model_selection import GridSearchCV

dt_regressor = DecisionTreeRegressor(random_state=42)
param_grid = {
    'max_depth': [None, 10, 20, 30],  # Maximum depth of the tree
    'min_samples_split': [2, 5, 10],  # Minimum number of samples required to split an internal node
    'min_samples_leaf': [1, 2, 4],  # Minimum number of samples required to be at a leaf node
}

grid_search = GridSearchCV(estimator=dt_regressor, param_grid=param_grid, scoring='neg_mean_squared_error', cv=5)
grid_search.fit(X_train_pca, y_train)

best_dt_params = grid_search.best_params_
best_dt_model = grid_search.best_estimator_
dt_feature_importance = best_dt_model.feature_importances_
dt_feature_importance_df = pd.DataFrame({'Feature': X_train_pca.columns, 'Importance': dt_feature_importance})


print("Best Hyperparameters:", best_dt_params)
print("Best Mean Squared Error:", abs(grid_search.best_score_))
print("\nDecisionTreeRegressor Feature Importance:")
print(dt_feature_importance_df)

In [ ]:
from sklearn.metrics import mean_squared_error,r2_score
y_pred = best_dt_model.predict(X_test_pca)
mse = mean_squared_error(y_test, y_pred)
print("Decision tree MSE:", mse)
r2score= r2_score(y_test, y_pred)
print("Decision tree R squared:",r2score)

In [ ]:
from sklearn.ensemble import RandomForestRegressor
rf_regressor = RandomForestRegressor(random_state=42)
param_grid_rf = {
    'n_estimators': [50, 100, 200],
    'max_depth': [10, 20, 30],
    'min_samples_split': [2, 5, 10],
    'min_samples_leaf': [1, 2, 4],
    'max_features':['sqrt', 'log2']
}

grid_search_rf = GridSearchCV(estimator=rf_regressor, param_grid=param_grid_rf, scoring='neg_mean_squared_error', cv=5)
grid_search_rf.fit(X_train_pca, y_train)
best_rf_model = grid_search_rf.best_estimator_
rf_feature_importance = best_rf_model.feature_importances_
rf_feature_importance_df = pd.DataFrame({'Feature': X_train_pca.columns, 'Importance': rf_feature_importance})



y_pred_rf = best_rf_model.predict(X_test_pca)
mse_rf = mean_squared_error(y_test, y_pred_rf)
print("Random Forest MSE:", mse_rf)
print("\nRandomForestRegressor Feature Importance:")
print(rf_feature_importance_df)

In [ ]:
#Ensembling using XGBoost regressor
from xgboost import XGBRegressor
xgb_regressor = XGBRegressor(random_state=42)

param_grid_xgb = {
    'n_estimators': [50, 100, 200],
    'learning_rate': [0.01, 0.1, 0.2],
    'max_depth': [3, 4, 5],
    'subsample': [0.8, 1.0],
    'colsample_bytree': [0.8, 1.0],
}

grid_search_xgb = GridSearchCV(estimator=xgb_regressor, param_grid=param_grid_xgb, scoring='neg_mean_squared_error', cv=5)
grid_search_xgb.fit(X_train, y_train)
best_xgb_model = grid_search_xgb.best_estimator_

y_pred_xgb = best_xgb_model.predict(X_tests)
mse_xgb = mean_squared_error(y_test, y_pred_xgb)
print("XGBoost MSE:", mse_xgb)
